# Imports

In [1]:
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, \
                         TrainingArguments, Trainer
from torch.optim import AdamW

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Process Dataset

In [2]:
raw_datasets = load_dataset("wnut_17")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
raw_datasets["train"][0]["tokens"]

['@paulwalk',
 'It',
 "'s",
 'the',
 'view',
 'from',
 'where',
 'I',
 "'m",
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'ESB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.']

In [5]:
raw_datasets["train"][0]["ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

In [7]:
label_names = ner_feature.feature.names
label_names

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [8]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

@paulwalk It 's the view from where I 'm living for two weeks . Empire     State      Building   = ESB        . Pretty bad storm here last evening . 
O         O  O  O   O    O    O     O O  O      O   O   O     O B-location I-location I-location O B-location O O      O   O     O    O    O       O 


In [9]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

In [11]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [12]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [14]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Setting Evaluation

In [16]:
metric = evaluate.load("seqeval")

In [17]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
# def compute_metrics_each_class(preds):
#     logits, labels = preds.predictions, pred.label_ids
#     predictions = np.argmax(logits, axis=-1)

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
#     return {
#         "precision": all_metrics["overall_precision"],
#         "recall": all_metrics["overall_recall"],
#         "f1": all_metrics["overall_f1"],
#         "accuracy": all_metrics["overall_accuracy"],
#     }

# Fine-tune Model

In [18]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

### Baseline model

In [20]:
args_baseline = TrainingArguments("bert-finetuned-baseline")
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

trainer_baseline = Trainer(
    model=model,
    args=args_baseline,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer_baseline.train()

test_pred_baseline = trainer_baseline.predict(test_dataset=tokenized_datasets["test"])
test_pred_baseline.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.116100
1000,0.044400


{'test_loss': 0.26021715998649597,
 'test_precision': 0.4791044776119403,
 'test_recall': 0.2974976830398517,
 'test_f1': 0.3670668953687822,
 'test_accuracy': 0.9334422110552764,
 'test_runtime': 103.7781,
 'test_samples_per_second': 12.401,
 'test_steps_per_second': 1.551}

### AdamW Optimization

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer1 = AdamW(model.parameters(), lr=2e-5)
args_adam1 = TrainingArguments(
    "bert-finetuned-adam1",
    evaluation_strategy="epoch",
)

trainer_adam1 = Trainer(
    model=model,
    args=args_adam1,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer1, None)
)

trainer_adam1.train()
test_pred_adam1 = trainer_adam1.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam1.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.167101,0.526126,0.349282,0.419842,0.915547
2,0.120500,0.180026,0.588138,0.427033,0.494802,0.922826
3,0.046500,0.173017,0.585227,0.492823,0.535065,0.927696


{'test_loss': 0.23202502727508545,
 'test_precision': 0.5274390243902439,
 'test_recall': 0.3206672845227062,
 'test_f1': 0.3988472622478386,
 'test_accuracy': 0.9359798994974874,
 'test_runtime': 10.1927,
 'test_samples_per_second': 126.267,
 'test_steps_per_second': 15.796}

In [19]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer6 = AdamW(model.parameters(), lr=2e-5)
args_adam6 = TrainingArguments(
    "bert-finetuned-adam1",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam6 = Trainer(
    model=model,
    args=args_adam6,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer6, None)
)

trainer_adam6.train()
test_pred_adam6 = trainer_adam6.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam6.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.214646,0.361538,0.056220,0.097308,0.888039
2,No log,0.188973,0.552301,0.315789,0.401826,0.910891
3,No log,0.179121,0.542808,0.379187,0.446479,0.916189


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.21806401014328003,
 'test_precision': 0.45422535211267606,
 'test_recall': 0.2391102873030584,
 'test_f1': 0.31329690346083794,
 'test_accuracy': 0.931180904522613,
 'test_runtime': 135.9734,
 'test_samples_per_second': 9.465,
 'test_steps_per_second': 1.184}

In [22]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer2 = AdamW(model.parameters(), lr=9e-5)
args_adam2 = TrainingArguments(
    "bert-finetuned-adam2",
    evaluation_strategy="epoch",
)

trainer_adam2 = Trainer(
    model=model,
    args=args_adam2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer2, None)
)

trainer_adam2.train()
test_pred_adam2 = trainer_adam2.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam2.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.154551,0.507553,0.401914,0.448598,0.919401
2,0.103700,0.199196,0.605405,0.401914,0.483106,0.917849
3,0.037400,0.211534,0.593005,0.446172,0.509215,0.922398


{'test_loss': 0.26479482650756836,
 'test_precision': 0.5007776049766719,
 'test_recall': 0.29842446709916587,
 'test_f1': 0.3739837398373984,
 'test_accuracy': 0.9330653266331659,
 'test_runtime': 10.885,
 'test_samples_per_second': 118.237,
 'test_steps_per_second': 14.791}

In [19]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer3 = AdamW(model.parameters(), lr=9e-5)
args_adam3 = TrainingArguments(
    "bert-finetuned-adam3",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam3 = Trainer(
    model=model,
    args=args_adam3,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer3, None)
)

trainer_adam3.train()
test_pred_adam3 = trainer_adam3.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam3.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.209008,0.561576,0.272727,0.367150,0.902542
2,No log,0.176218,0.618861,0.376794,0.468401,0.916082
3,No log,0.186741,0.523529,0.425837,0.469657,0.920203


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.23311103880405426,
 'test_precision': 0.4780701754385965,
 'test_recall': 0.3030583873957368,
 'test_f1': 0.3709585933068633,
 'test_accuracy': 0.9337437185929648,
 'test_runtime': 135.5531,
 'test_samples_per_second': 9.494,
 'test_steps_per_second': 1.188}

In [22]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer4 = AdamW(model.parameters(), lr=1e-5)
args_adam4 = TrainingArguments(
    "bert-finetuned-adam4",
    evaluation_strategy="epoch",
)

trainer_adam4 = Trainer(
    model=model,
    args=args_adam4,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer4, None)
)

trainer_adam4.train()
test_pred_adam4 = trainer_adam4.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam4.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.187251,0.568627,0.312201,0.403089,0.906074
2,0.147700,0.190763,0.590998,0.361244,0.448404,0.914209
3,0.064700,0.177316,0.579872,0.434211,0.496580,0.920524


{'test_loss': 0.23042526841163635,
 'test_precision': 0.4959349593495935,
 'test_recall': 0.28266913809082483,
 'test_f1': 0.3600944510035419,
 'test_accuracy': 0.9325125628140704,
 'test_runtime': 10.5206,
 'test_samples_per_second': 122.332,
 'test_steps_per_second': 15.303}

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer5 = AdamW(model.parameters(), lr=1e-5)
args_adam5 = TrainingArguments(
    "bert-finetuned-adam5",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam5 = Trainer(
    model=model,
    args=args_adam5,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer5, None)
)

trainer_adam5.train()
test_pred_adam5 = trainer_adam5.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam5.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.244312,0.000000,0.000000,0.000000,0.881777
2,No log,0.220559,0.483871,0.089713,0.151362,0.890072
3,No log,0.212262,0.537313,0.172249,0.260870,0.894461


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.23291215300559998,
 'test_precision': 0.507177033492823,
 'test_recall': 0.09823911028730306,
 'test_f1': 0.1645962732919255,
 'test_accuracy': 0.9203517587939698,
 'test_runtime': 10.3259,
 'test_samples_per_second': 124.639,
 'test_steps_per_second': 15.592}